In [1]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
ticker = 'AAPL'
data = yf.download(ticker)
# Display the first few rows of the dataset
print(data.head())

In [3]:
# Calculate moving averages
data['MA_10'] = data['Close'].rolling(window=10).mean()
data['MA_50'] = data['Close'].rolling(window=50).mean()

# Drop NaN values
data = data.dropna()

# Define features and target
X = data[['Close', 'MA_10', 'MA_50']]
y = data['Close'].shift(-1).dropna()  # Shift target by 1 day to predict next day's close price
X = X[:-1]  # Align the features with the target

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [ ]:
# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')

In [ ]:
# Ensure the index is sorted to match the time series order for plotting
plt.figure(figsize=(14, 7))

# Sort the data before plotting
plt.plot(y_test.index, y_test.values, label='Actual Price')
plt.plot(y_test.index, predictions, label='Predicted Price')

# Adding labels and title
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Actual vs. Predicted Stock Prices')
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Set initial balance and position
initial_balance = 10000  # Starting balance in USD
balance = initial_balance
position = 0  # Number of shares held

# Simulate trading over X_test
for i in range(len(X_test)):
    current_price = X_test.iloc[i]['Close']  # Current stock price (scalar)
    predicted_price = predictions[i]  # Predicted stock price for the next day (scalar)

    # Ensure both are scalar values
    current_price = current_price.item() if isinstance(current_price, pd.Series) else current_price
    predicted_price = predicted_price.item() if isinstance(predicted_price, pd.Series) else predicted_price

    # Buy condition: predicted price is higher than current price and we have enough balance
    if predicted_price > current_price and balance >= current_price:
        # Buy stock (whole shares only)
        shares_to_buy = int(balance // current_price)
        if shares_to_buy > 0:
            position += shares_to_buy  # Increase number of shares held
            balance -= shares_to_buy * current_price  # Deduct the cost from balance
            print(f"Buying {shares_to_buy} shares at {current_price:.2f} | New balance: ${balance:.2f}")

    # Sell condition: predicted price is lower than current price and we have shares to sell
    elif predicted_price < current_price and position > 0:
        # Sell all shares
        balance += position * current_price  # Add sale amount to balance
        print(f"Selling {position} shares at {current_price:.2f} | New balance: ${balance:.2f}")
        position = 0  # Reset position after selling all shares

# After loop ends, sell remaining shares (if any) at the last known price
if position > 0:
    remaining_price = X_test.iloc[-1]['Close'].item()  # Extract the scalar value
    balance += position * remaining_price  # Sell remaining shares at last price
    print(f"Selling remaining {position} shares at {remaining_price:.2f}")  # Properly format the scalar value
    position = 0

# Final balance calculation (including the remaining position value, if any)
final_balance = balance
profit = final_balance - initial_balance

# Output the final results
print(f"Final balance: ${final_balance:.2f}")
print(f"Profit: ${profit:.2f}")

